## **Projet 1 : Fine-tuning d’un Modèle LLAMA 3**  

### 🎯 **Objectif**  
L’objectif de cet exercice est de **reprendre toutes les étapes** du notebook pour fine-tuner un modèle **LLAMA 3**, afin qu’il puisse répondre à la question :  
**"Quelles sont les champs d'activité de l'INRAE ?"**  

Vous devrez :   
✅ **Télécharger et préparer le modèle LLAMA 3**  
✅ **Créer et pré-traiter un dataset spécifique**  
✅ **Effectuer le fine-tuning avec LoRA**  
✅ **Sauvegarder le modèle entraîné**  
✅ **Convertir le modèle en format GGUF pour Ollama**  
✅ **Tester le modèle en lui posant la question cible**  

In [13]:
# Import du modèle depuis hugging face

model_name = "meta-llama/Llama-3.2-1B-Instruct"

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

In [14]:
# Traitement du jeu de données
from datasets import load_dataset

file_path = './data.jsonl'

# ✅ Charger le dataset JSONL
dataset = load_dataset('json', data_files=file_path)


# Prétraitement des données d'entrainement
def preprocess_function(examples):
    # 🔹 Construire le texte d'entrée à partir des messages
    text_inputs = []
    for message_set in examples["messages"]:
        text = ""
        for message in message_set:
            text += f"{message['content']}"
        text_inputs.append(text.strip())

    # 🔹 Tokenisation
    inputs = tokenizer(text_inputs, truncation=True, padding="max_length", max_length=25)

    # 🔹 Copier input_ids pour labels
    inputs["labels"] = inputs["input_ids"].copy()

    # 🔹 Remplacer les tokens de padding par -100 pour ignorer la perte
    padding_token_id = tokenizer.pad_token_id
    inputs["labels"] = [
        [(label if label != padding_token_id else -100) for label in labels] for labels in inputs["labels"]
    ]

    return inputs

# ✅ Appliquer la transformation
dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 4/4 [00:00<00:00, 40.43 examples/s]


In [15]:
# Entrainement du modèle

n_epoches = 10

from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq


# ✅ Config LoRA
lora_config = LoraConfig(
    r=32, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)


print("---------------------------------------------------------------")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("---------------------------------------------------------------")

training_args = TrainingArguments(
    output_dir="./llama3-finetuned",
    per_device_train_batch_size=5,  # ⚠️ Réduire la batch size car CPU est limité
    num_train_epochs=n_epoches,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-3,
    gradient_accumulation_steps=90,  # 🔹 Augmenter pour réduire la charge mémoire
    fp16=False,  # 🚫 Désactiver fp16 (inutile sur CPU)
    bf16=False,
    gradient_checkpointing=False,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    save_total_limit=2,
    weight_decay=0.01,
    report_to="tensorboard",
    torch_compile=False,  # ✅ Optimisation CPU
    no_cuda=True
)


# ✅ Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt")


# ✅ Entraînement
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['train'],
    data_collator=data_collator
)

trainer.train()

---------------------------------------------------------------
trainable params: 6,815,744 || all params: 1,242,630,144 || trainable%: 0.5485
---------------------------------------------------------------


c:\Users\Quera\Desktop\INRAE-Llama3-main\.llama_env\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\Quera\Desktop\INRAE-Llama3-main\.llama_env\Lib\site-packages\transformers\training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,3.148244
2,No log,3.028309
3,No log,2.844520
4,No log,2.644104
5,No log,2.434597
6,No log,2.194132
7,No log,1.943145
8,No log,1.699063
9,No log,1.450433
10,No log,1.103003


TrainOutput(global_step=10, training_loss=2.3511348724365235, metrics={'train_runtime': 132.6604, 'train_samples_per_second': 0.302, 'train_steps_per_second': 0.075, 'total_flos': 7526107054080.0, 'train_loss': 2.3511348724365235, 'epoch': 10.0})

In [ ]:
# Sauvegarde du modèle
# Sauvegarde du modèle LoRA et fusion des poids du modèle de base
from transformers import AutoModelForCausalLM
from peft import PeftModel

model.save_pretrained("llama3-finetuned")
tokenizer.save_pretrained("llama3-finetuned")


base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", device_map="cpu")
peft_model = PeftModel.from_pretrained(base_model, "llama3-finetuned")


# Fusionner les poids LoRA avec le modèle principal
peft_model = peft_model.merge_and_unload()



# Sauvegarde du modèle fusionné (sans LoRA)
peft_model.save_pretrained("llama3-finetuned-merged")
tokenizer.save_pretrained("llama3-finetuned-merged")

print("✅ Fusion et sauvegarde du modèle terminé !")



In [ ]:
# Convertion du modèle avec CCP et Ollama
!python ./llama.cpp/convert_hf_to_gguf.py ./llama3-finetuned-merged

!ollama create llama-INRAE -f Modelfile


In [19]:
!python fine-tuning.py "meta-llama/Llama-3.2-1B-Instruct" './data.jsonl' 10

^C


Traceback (most recent call last):
  File "c:\Users\Quera\Desktop\INRAE-Llama3-main\fine-tuning.py", line 22, in <module>
    dataset = load_dataset('json', data_files=file_path)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Quera\Desktop\INRAE-Llama3-main\.llama_env\Lib\site-packages\datasets\load.py", line 2129, in load_dataset
    builder_instance = load_dataset_builder(
                       ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Quera\Desktop\INRAE-Llama3-main\.llama_env\Lib\site-packages\datasets\load.py", line 1849, in load_dataset_builder
    dataset_module = dataset_module_factory(
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Quera\Desktop\INRAE-Llama3-main\.llama_env\Lib\site-packages\datasets\load.py", line 1564, in dataset_module_factory
    ).get_module()
      ^^^^^^^^^^^^
  File "c:\Users\Quera\Desktop\INRAE-Llama3-main\.llama_env\Lib\site-packages\datasets\load.py", line 944, in get_module
    data_files = DataFilesDict.from

['fine-tuning.py', 'meta-llama/Llama-3.2-1B-Instruct', "'./data.jsonl'", '10']


In [29]:
import os

os.system("python ./llama.cpp/convert_hf_to_gguf.py ./llama3-finetuned-merged")
os.system("ollama create llama-INRAE -f Modelfile")



0